# Overview

The python package `pyvistaqt` extends the functionality of `pyvista` through the usage of Qt. Since Qt applications operates in a separate thread than VTK, you can simultaneously have an active VTK plot and a non-blocking Python session.

![qt_multiplot](https://qtdocs.pyvista.org/_images/qt_multiplot.png)

# Getting Started

In [ ]:
!pip install pyqt5

Installation using `pip` is:

In [ ]:
!pip install pyvistaqt

To install this package with conda run:

```bash
$ conda install -c conda-forge pyvistaqt
```

You can also visit [PyPI](https://pypi.org/project/pyvistaqt/) or [GitHub](https://github.com/pyvista/pyvistaqt) to download the source.

Once installed, use the [`pyvistaqt.BackgroundPlotter`](https://qtdocs.pyvista.org/api_reference.html#pyvistaqt.BackgroundPlotter) like any PyVista plotter.

# Brief Example

Create an instance of the [pyvistaqt.BackgroundPlotter](https://qtdocs.pyvista.org/api_reference.html#pyvistaqt.BackgroundPlotter) and plot a sphere.

In [ ]:
import pyvista as pv
from pyvistaqt import BackgroundPlotter

sphere = pv.Sphere()

plotter = BackgroundPlotter()
plotter.add_mesh(sphere)

# Usage

PyVista has an interface for placing plots in `pyvistaqt` that extends the functionality of the `QVTKRenderWindowInteractor` class. The `pyvistaqt.QtInteractor` class allows you to have the same functionality of the `Plotter` class within a Qt application. This simplifies adding meshes, updating, and controlling them when using Qt.

Please do keep in mind that the `BackgroundPlotter` **does not** create its own event loop by default. By design, the plotter will look for an active instance of `QApplication` instead. So in the end, it is up to the user to manage this event loop and there are several ways to achieve this. For example, it’s possible to start Python interactively with `python -i`, use `ipython` or execute the Qt event loop by adding `plotter.app.exec_()` to the end of the following code.

# Background Plotting

Normal PyVista plotting windows exhibit blocking behavior, but it is possible to plot in the background and update the plotter in real-time using the `BackgroundPlotter` object. This requires `pyvistaqt`, but otherwise appears and functions like a normal PyVista `Plotter` instance. For example:

In [ ]:
import pyvista as pv
from pyvistaqt import BackgroundPlotter

sphere = pv.Sphere()

plotter = BackgroundPlotter()
plotter.add_mesh(sphere)

# can now operate on the sphere and have it updated in the background
sphere.points *= 0.5

# Multiple Plotters

The following example shows how to use an interface with multiple plotters. Each plotter can be selected and functions like a normal PyVista `Plotter` instance:

In [ ]:
import pyvista as pv
from pyvistaqt import MultiPlotter

mp = MultiPlotter(nrows=2, ncols=2)
mp[0, 0].add_mesh(pv.Sphere())
mp[0, 1].add_mesh(pv.Cylinder())
mp[1, 0].add_mesh(pv.Cube())
mp[1, 1].add_mesh(pv.Cone())

# Example PyQt5 PyVista QtInteractor

The following example shows how to create a simple application that adds a sphere to an empty plotting window.

In [ ]:
import sys

# Setting the Qt bindings for QtPy
import os
os.environ["QT_API"] = "pyqt5"

from qtpy import QtWidgets

import numpy as np

import pyvista as pv
from pyvistaqt import QtInteractor, MainWindow

class MyMainWindow(MainWindow):

    def __init__(self, parent=None, show=True):
        QtWidgets.QMainWindow.__init__(self, parent)

        # create the frame
        self.frame = QtWidgets.QFrame()
        vlayout = QtWidgets.QVBoxLayout()

        # add the pyvista interactor object
        self.plotter = QtInteractor(self.frame)
        vlayout.addWidget(self.plotter.interactor)
        self.signal_close.connect(self.plotter.close)

        self.frame.setLayout(vlayout)
        self.setCentralWidget(self.frame)

        # simple menu to demo functions
        mainMenu = self.menuBar()
        fileMenu = mainMenu.addMenu('File')
        exitButton = QtWidgets.QAction('Exit', self)
        exitButton.setShortcut('Ctrl+Q')
        exitButton.triggered.connect(self.close)
        fileMenu.addAction(exitButton)

        # allow adding a sphere
        meshMenu = mainMenu.addMenu('Mesh')
        self.add_sphere_action = QtWidgets.QAction('Add Sphere', self)
        self.add_sphere_action.triggered.connect(self.add_sphere)
        meshMenu.addAction(self.add_sphere_action)

        if show:
            self.show()

    def add_sphere(self):
        """ add a sphere to the pyqt frame """
        sphere = pv.Sphere()
        self.plotter.add_mesh(sphere, show_edges=True)
        self.plotter.reset_camera()


if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    window = MyMainWindow()
    sys.exit(app.exec_())

# Using Different Qt bindings

To use different Qt bindings you must first install them. For example, to use `PySide2`, you install it via:

In [ ]:
!pip install PySide2

Then you set the `QT_API` value to the specific binding you would like to use:

In [ ]:
os.environ["QT_API"] = "pyside2"

Please refer to the [*QtPy* documentation](https://github.com/spyder-ide/qtpy) page for more information.

### Excersize

Let's plot globe using `BackgroundPlotter` object.

In [ ]:
# Your code here

In [ ]:
from pyvista import examples
import pyvista as pv
from pyvistaqt import BackgroundPlotter


globe = examples.load_globe()

plotter = BackgroundPlotter()
plotter.add_mesh(globe)

# Freezing PyVista with pyinstaller

You can make some fantastic standalone programs with `pyinstaller` and `pyvista`, and you can even make a graphical user interface incorporating `PyQt5` or `pyside2`. Depending on your version of VTK, this requires some extra steps to setup.

When running VTK v9, you need to add several additional `hiddenimports`. For clarity and completeness, create a spec file (we’ll name it `pyvista.spec`) following the directions given at [Using Spec Files](https://pyinstaller.readthedocs.io/en/stable/spec-files.html). Modify the `Analysis` and add the following hidden imports:

```python
main_py = os.path.join(some_path, 'main.py')
a = Analysis([main_py],
             pathex=[],
             binaries=[],
             hiddenimports=['vtkmodules',
                            'vtkmodules.all',
                            'vtkmodules.qt.QVTKRenderWindowInteractor',
                            'vtkmodules.util',
                            'vtkmodules.util.numpy_support',
                            'vtkmodules.numpy_interface.dataset_adapter',
                           ],
```

# Flask Application

You can use `pyvista` in to make a flask application to display static plots. See the following example as well as the demo at [Flask Example](https://github.com/pyvista/pyvista/tree/main/examples_flask).

For dynamic examples, it’s recommended to use [Jupyter Notebooks](https://jupyter.org/). See our documentation regarding this at [Jupyter Notebook Plotting](https://docs.pyvista.org/user-guide/jupyter/index.html#jupyter-plotting).

![flask example](https://docs.pyvista.org/_images/flask_example.png)

# Python Application app.py

In [ ]:
%load examples_flask/static_ex/app.py

In [ ]:
!cd examples_flask/static_ex && python app.py

# Ajax Template index.html

This template should be within the `templates` directory in the same path as `app.py`.

This template returns the `meshtype` parameter back to the `get_img` method in the flask app, which is used to select the type of mesh to be plotted.

In [ ]:
%load examples_flask/static_ex/templates/index.html

# PyVista within a Docker Container

You can use `pyvista` from within a docker container with jupyterlab. To launch a local docker container, install `docker`, then pull and run the image with:

Finally, open the link that shows up from the terminal output and start playing around with pyvista in jupyterlab! For example:

You can see the latest tags of [our Docker containers here](https://github.com/pyvista/pyvista/pkgs/container/pyvista). `ghcr.io/pyvista/pyvista:latest` has JupyterLab set up while `ghcr.io/pyvista/pyvista:latest-slim` is a lightweight Python environment without Jupyter

You may need to log into the GitHub container registry by following the directions at [Working with the Docker registry](https://docs.github.com/en/enterprise-server@3.0/packages/working-with-a-github-packages-registry/working-with-the-docker-registry))

# Create your own Docker Container with pyvista

Clone pyvista and cd into this directory to create your own customized docker image.

```bash
git clone https://github.com/pyvista/pyvista
cd pyvista/docker
IMAGE=my-pyvista-jupyterlab:v0.1.0
docker build -t $IMAGE .
docker push $IMAGE
```

If you wish to have off-screen GPU support when rending on jupyterlab, see the the notes about building with EGL at [Building VTK](https://docs.pyvista.org/extras/building_vtk.html#ref-building-vtk), or use the custom, pre-built wheels at [Release 0.27.0](https://github.com/pyvista/pyvista/releases/tag/0.27.0). Install that customized vtk wheel onto your docker image by modifying the docker image at `pyvista/docker/jupyter.Dockerfile` with:

Additionally, you must install GPU drivers on the docker image of the same version running on the host machine. For example, if you are running on Azure Kubernetes Service and the GPU nodes on the kubernetes cluster are running `450.51.06`, you must install the same version on your image. Since you will be using the underlying kernel module, there’s no reason to build it on the container (and trying will only result in an error).

To verify that you’re rendering on a GPU, first check the output of `nvidia-smi`. You should get something like:

```bash
$ nvidia-smi
Sun Nov  8 05:48:46 2020
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000001:00:00.0 Off |                    0 |
| N/A   34C    P8    32W / 149W |   1297MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
```

Note the driver version (which is actually the kernel driver version), and verify it matches the version you installed on your docker image.

Finally, check that your render window is using NVIDIA by running `ReportCapabilities`:

In [ ]:
import pyvista
pl = pyvista.Plotter()
print(pl.ren_win.ReportCapabilities())

If you get `display id not set`, then your environment is likely not set up correctly.

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Box Widget {#box_widget_example}
==========

The box widget can be enabled and disabled by the
`pyvista.WidgetHelper.add_box_widget`{.interpreted-text role="func"} and
`pyvista.WidgetHelper.clear_box_widgets`{.interpreted-text role="func"}
methods respectively. When enabling the box widget, you must provide a
custom callback function otherwise the box would appear and do nothing -
the callback functions are what allow us to leverage the widget to
perform a task like clipping/cropping.

Considering that using a box to clip/crop a mesh is one of the most
common use cases, we have included a helper method that will allow you
to add a mesh to a scene with a box widget that controls its extent, the
`pyvista.WidgetHelper.add_mesh_clip_box`{.interpreted-text role="func"}
method.

![image](../../images/gifs/box-clip.gif)


In [ ]:
import pyvista as pv
from pyvista import examples

mesh = examples.download_nefertiti()

In [ ]:
p = pv.Plotter()
p.add_mesh_clip_box(mesh, color='white')
p.show(cpos=[-1, -1, 0.2])

After interacting with the scene, the clipped mesh is available as:


In [ ]:
p.box_clipped_meshes

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Checkbox Widget
===============

Use a checkbox to turn on/off the visibility of meshes in a scene.

See `pyvista.WidgetHelper.add_checkbox_button_widget`{.interpreted-text
role="func"} for more details.


In [ ]:
import pyvista as pv

Single Checkbox
===============


In [ ]:
mesh = pv.Sphere()

p = pv.Plotter()
actor = p.add_mesh(mesh)


def toggle_vis(flag):
    actor.SetVisibility(flag)


p.add_checkbox_button_widget(toggle_vis, value=True)
p.show()

Multiple Checkboxes
===================

In this example, we will add many meshes to a scene with unique colors
and create corresponding checkboxes for those meshes of the same color
to toggle their visibility in the scene.


In [ ]:
colors = [
    ["ff0000", "28e5da", "0000ff"],
    ["ffff00", "c8bebe", "f79292"],
    ["fffff0", "f18c1d", "23dcaa"],
    ["d785ec", "9d5b13", "e4e0b1"],
    ["894509", "af45f5", "fff000"],
]


class SetVisibilityCallback:
    """Helper callback to keep a reference to the actor being modified."""

    def __init__(self, actor):
        self.actor = actor

    def __call__(self, state):
        self.actor.SetVisibility(state)

In [ ]:
# Widget size
size = 50

p = pv.Plotter()

Startpos = 12
for i, lst in enumerate(colors):
    for j, color in enumerate(lst):
        actor = p.add_mesh(pv.Sphere(center=(i, j, 0)), color=color)
        # Make a separate callback for each widget
        callback = SetVisibilityCallback(actor)
        p.add_checkbox_button_widget(
            callback,
            value=True,
            position=(5.0, Startpos),
            size=size,
            border_size=1,
            color_on=color,
            color_off='grey',
            background_color='grey',
        )
        Startpos = Startpos + size + (size // 10)

p.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Line Widget {#line_widget_example}
===========

The line widget can be enabled and disabled by the
`pyvista.WidgetHelper.add_line_widget`{.interpreted-text role="func"}
and `pyvista.WidgetHelper.clear_line_widgets`{.interpreted-text
role="func"} methods respectively. Unfortunately, PyVista does not have
any helper methods to utilize this widget, so it is necessary to pass a
custom callback method.

One particularly fun example is to use the line widget to create a
source for the `pyvista.DataSetFilters.streamlines`{.interpreted-text
role="func"} filter. Again note the use of the `name` argument in
`add_mesh`.


In [ ]:
import numpy as np

import pyvista as pv
from pyvista import examples

pv.set_plot_theme('document')

mesh = examples.download_kitchen()
furniture = examples.download_kitchen(split=True)

arr = np.linalg.norm(mesh['velocity'], axis=1)
clim = [arr.min(), arr.max()]

In [ ]:
p = pv.Plotter()
p.add_mesh(furniture, name='furniture', color=True)
p.add_mesh(mesh.outline(), color='black')
p.add_axes()


def simulate(pointa, pointb):
    streamlines = mesh.streamlines(
        n_points=10, max_steps=100, pointa=pointa, pointb=pointb, integration_direction='forward'
    )
    p.add_mesh(streamlines, name='streamlines', line_width=5, render_lines_as_tubes=True, clim=clim)


p.add_line_widget(callback=simulate, use_vertices=True)
p.show()

And here is a screen capture of a user interacting with this

![image](../../images/gifs/line-widget-streamlines.gif)


In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Multiple Slider Widgets
=======================

Use a class based callback to track multiple slider widgets for updating
a single mesh.

In this example we simply change a few parameters for the
`pyvista.Sphere`{.interpreted-text role="func"} method, but this could
easily be applied to any mesh-generating/altering code.


In [ ]:
import pyvista as pv


class MyCustomRoutine:
    def __init__(self, mesh):
        self.output = mesh  # Expected PyVista mesh type
        # default parameters
        self.kwargs = {
            'radius': 0.5,
            'theta_resolution': 30,
            'phi_resolution': 30,
        }

    def __call__(self, param, value):
        self.kwargs[param] = value
        self.update()

    def update(self):
        # This is where you call your simulation
        result = pv.Sphere(**self.kwargs)
        self.output.overwrite(result)
        return

In [ ]:
starting_mesh = pv.Sphere()
engine = MyCustomRoutine(starting_mesh)

In [ ]:
p = pv.Plotter()
p.add_mesh(starting_mesh, show_edges=True)
p.add_slider_widget(
    callback=lambda value: engine('phi_resolution', int(value)),
    rng=[3, 60],
    value=30,
    title="Phi Resolution",
    pointa=(0.025, 0.1),
    pointb=(0.31, 0.1),
    style='modern',
)
p.add_slider_widget(
    callback=lambda value: engine('theta_resolution', int(value)),
    rng=[3, 60],
    value=30,
    title="Theta Resolution",
    pointa=(0.35, 0.1),
    pointb=(0.64, 0.1),
    style='modern',
)
p.add_slider_widget(
    callback=lambda value: engine('radius', value),
    rng=[0.1, 1.5],
    value=0.5,
    title="Radius",
    pointa=(0.67, 0.1),
    pointb=(0.98, 0.1),
    style='modern',
)
p.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Plane Widget {#plane_widget_example}
============

The plane widget can be enabled and disabled by the
`pyvista.WidgetHelper.add_plane_widget`{.interpreted-text role="func"}
and `pyvista.WidgetHelper.clear_plane_widgets`{.interpreted-text
role="func"} methods respectively. As with all widgets, you must provide
a custom callback method to utilize that plane. Considering that planes
are most commonly used for clipping and slicing meshes, we have included
two helper methods for doing those tasks!

Let\'s use a plane to clip a mesh:


In [ ]:
import pyvista as pv
from pyvista import examples

vol = examples.download_brain()

p = pv.Plotter()
p.add_mesh_clip_plane(vol)
p.show()

After interacting with the scene, the clipped mesh is available as:


In [ ]:
p.plane_clipped_meshes

And here is a screen capture of a user interacting with this

![image](../../images/gifs/plane-clip.gif)


Or you could slice a mesh using the plane widget:


In [ ]:
p = pv.Plotter()
p.add_mesh_slice(vol)
p.show()

After interacting with the scene, the slice is available as:


In [ ]:
p.plane_sliced_meshes

And here is a screen capture of a user interacting with this

![image](../../images/gifs/plane-slice.gif)


Or you could leverage the plane widget for some custom task like
glyphing a vector field along that plane. Note that we have to pass a
`name` when calling `add_mesh` to ensure that there is only one set of
glyphs plotted at a time.


In [ ]:
import pyvista as pv
from pyvista import examples

mesh = examples.download_carotid()

p = pv.Plotter()
p.add_mesh(mesh.contour(8).extract_largest(), opacity=0.5)


def my_plane_func(normal, origin):
    slc = mesh.slice(normal=normal, origin=origin)
    arrows = slc.glyph(orient='vectors', scale="scalars", factor=0.01)
    p.add_mesh(arrows, name='arrows')


p.add_plane_widget(my_plane_func)
p.show_grid()
p.add_axes()
p.show()

And here is a screen capture of a user interacting with this

![image](../../images/gifs/plane-glyph.gif)


Further, a user can disable the arrow vector by setting the
`normal_rotation` argument to `False`. For example, here we
programmatically set the normal vector on which we want to translate the
plane and we disable the arrow to prevent its rotation.


In [ ]:
p = pv.Plotter()
p.add_mesh_slice(vol, normal=(1, 1, 1), normal_rotation=False)
p.show()

The vector is also forcibly disabled anytime the `assign_to_axis`
argument is set.


In [ ]:
p = pv.Plotter()
p.add_mesh_slice(vol, assign_to_axis='z')
p.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Slider Bar Widget {#slider_bar_widget_example}
=================

The slider widget can be enabled and disabled by the
`pyvista.WidgetHelper.add_slider_widget`{.interpreted-text role="func"}
and `pyvista.WidgetHelper.clear_slider_widgets`{.interpreted-text
role="func"} methods respectively. This is one of the most versatile
widgets as it can control a value that can be used for just about
anything.


One helper method we\'ve added is the
`pyvista.WidgetHelper.add_mesh_threshold`{.interpreted-text role="func"}
method which leverages the slider widget to control a thresholding
value.


In [ ]:
import pyvista as pv
from pyvista import examples

mesh = examples.download_knee_full()

p = pv.Plotter()
p.add_mesh_threshold(mesh)
p.show()

After interacting with the scene, the threshold mesh is available as:


In [ ]:
p.threshold_meshes

And here is a screen capture of a user interacting with this

![image](../../images/gifs/slider-widget-threshold.gif)


Custom Callback
===============

Or you could leverage a custom callback function that takes a single
value from the slider as its argument to do something like control the
resolution of a mesh. Again note the use of the `name` argument in
`add_mesh`:


In [ ]:
p = pv.Plotter()


def create_mesh(value):
    res = int(value)
    sphere = pv.Sphere(phi_resolution=res, theta_resolution=res)
    p.add_mesh(sphere, name='sphere', show_edges=True)
    return


p.add_slider_widget(create_mesh, [5, 100], title='Resolution')
p.show()

And here is a screen capture of a user interacting with this

![image](../../images/gifs/slider-widget-resolution.gif)


In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Sphere Widget
=============

The sphere widget can be enabled and disabled by the
`pyvista.WidgetHelper.add_sphere_widget`{.interpreted-text role="func"}
and `pyvista.WidgetHelper.clear_sphere_widgets`{.interpreted-text
role="func"} methods respectively. This is a very versatile widget as it
can control vertex location that can be used to control or update the
location of just about anything.

We don\'t have any convenient helper methods that utilize this widget
out of the box, but we have added a lot of ways to use this widget so
that you can easily add several widgets to a scene.

Let\'s look at a few use cases that all update a surface mesh.


Example A
=========

Use a single sphere widget


In [ ]:
import numpy as np

import pyvista as pv

# Create a triangle surface
surf = pv.PolyData()
surf.points = np.array(
    [
        [-10, -10, -10],
        [10, 10, -10],
        [-10, 10, 0],
    ]
)
surf.faces = np.array([3, 0, 1, 2])

p = pv.Plotter()


def callback(point):
    surf.points[0] = point


p.add_sphere_widget(callback)
p.add_mesh(surf, color=True)

p.show_grid()
p.show()

And here is a screen capture of a user interacting with this

![image](../../images/gifs/sphere-widget-a.gif)


Example B
=========

Use several sphere widgets at once


In [ ]:
import numpy as np

import pyvista as pv

# Create a triangle surface
surf = pv.PolyData()
surf.points = np.array(
    [
        [-10, -10, -10],
        [10, 10, -10],
        [-10, 10, 0],
    ]
)
surf.faces = np.array([3, 0, 1, 2])


p = pv.Plotter()


def callback(point, i):
    surf.points[i] = point


p.add_sphere_widget(callback, center=surf.points)
p.add_mesh(surf, color=True)

p.show_grid()
p.show()

And here is a screen capture of a user interacting with this

![image](../../images/gifs/sphere-widget-b.gif)


Example C
=========

This one is the coolest - use four sphere widgets to update
perturbations on a surface and interpolate between them with some
boundary conditions


In [ ]:
import numpy as np
from scipy.interpolate import griddata

import pyvista as pv


def get_colors(n):
    """A helper function to get n colors"""
    from itertools import cycle

    import matplotlib

    cycler = matplotlib.rcParams['axes.prop_cycle']
    colors = cycle(cycler)
    colors = [next(colors)['color'] for i in range(n)]
    return colors


# Create a grid to interpolate to
xmin, xmax, ymin, ymax = 0, 100, 0, 100
x = np.linspace(xmin, xmax, num=25)
y = np.linspace(ymin, ymax, num=25)
xx, yy, zz = np.meshgrid(x, y, [0])

# Make sure boundary conditions exist
boundaries = np.array([[xmin, ymin, 0], [xmin, ymax, 0], [xmax, ymin, 0], [xmax, ymax, 0]])

# Create the PyVista mesh to hold this grid
surf = pv.StructuredGrid(xx, yy, zz)

# Create some initial perturbations
# - this array will be updated inplace
points = np.array([[33, 25, 45], [70, 80, 13], [51, 57, 10], [25, 69, 20]])


# Create an interpolation function to update that surface mesh
def update_surface(point, i):
    points[i] = point
    tp = np.vstack((points, boundaries))
    zz = griddata(tp[:, 0:2], tp[:, 2], (xx[:, :, 0], yy[:, :, 0]), method='cubic')
    surf.points[:, -1] = zz.ravel(order='F')
    return


# Get a list of unique colors for each widget
colors = get_colors(len(points))

In [ ]:
# Begin the plotting routine
p = pv.Plotter()

# Add the surface to the scene
p.add_mesh(surf, color=True)

# Add the widgets which will update the surface
p.add_sphere_widget(update_surface, center=points, color=colors, radius=3)
# Add axes grid
p.show_grid()

# Show it!
p.show()

And here is a screen capture of a user interacting with this

![image](../../images/gifs/sphere-widget-c.gif)


In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Spline Widget
=============

A spline widget can be enabled and disabled by the
`pyvista.WidgetHelper.add_spline_widget`{.interpreted-text role="func"}
and `pyvista.WidgetHelper.clear_spline_widgets`{.interpreted-text
role="func"} methods respectively. This widget allows users to
interactively create a poly line (spline) through a scene and use that
spline.

A common task with splines is to slice a volumetric dataset using an
irregular path. To do this, we have added a convenient helper method
which leverages the
`pyvista.DataSetFilters.slice_along_line`{.interpreted-text role="func"}
filter named
`pyvista.WidgetHelper.add_mesh_slice_spline`{.interpreted-text
role="func"}.


In [ ]:
import numpy as np

import pyvista as pv

In [ ]:
mesh = pv.Wavelet()

# initial spline to seed the example
points = np.array(
    [
        [-8.64208925, -7.34294559, -9.13803458],
        [-8.25601497, -2.54814702, 0.93860914],
        [-0.30179377, -3.21555997, -4.19999019],
        [3.24099167, 2.05814768, 3.39041509],
        [4.39935227, 4.18804542, 8.96391132],
    ]
)

p = pv.Plotter()
p.add_mesh(mesh.outline(), color='black')
p.add_mesh_slice_spline(mesh, initial_points=points, n_handles=5)
p.camera_position = [(30, -42, 30), (0.0, 0.0, 0.0), (-0.09, 0.53, 0.84)]
p.show()